In [1]:
import pandas as pd
import numpy as np
import re
import torch
import json
from tqdm.notebook import tqdm
# from torch.utils.data import Dataset, DataLoader
# from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from sklearn.model_selection import train_test_split

In [2]:
import sys

sys.path.append('../utils')
from evaluator import Evaluator
from json_format import SepTokenJSONProcessor

json_proc = SepTokenJSONProcessor()

In [3]:
train = pd.read_csv('../data/train_9k_valid.csv', index_col=0, converters={'json': json.loads}).sample(frac=1, random_state=42)
val_set = pd.read_csv('../data/val_set_300_sb_valid.csv', index_col=0, converters={'json': json.loads}).sample(frac=1, random_state=42)
manual_test = pd.read_csv('../data/manual_test_100.csv', index_col=0, converters={'json': json.loads}).sample(frac=1, random_state=42)

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-large")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
tokenizer.add_tokens(json_proc.spec_tokens)
model.resize_token_embeddings(len(tokenizer))


assert train.loc[train.index[0], 'json'] == json_proc.unprocess_json(json_proc.process_json(train.loc[train.index[0], 'json']))

In [6]:
from datasets import Dataset

ads_dataset = Dataset.from_pandas(train[["Text", "json"]])
ads_dataset = ads_dataset.train_test_split(test_size=0.025, seed=42)
ads_dataset = ads_dataset.flatten()
ads_dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'json', '__index_level_0__'],
        num_rows: 8590
    })
    test: Dataset({
        features: ['Text', 'json', '__index_level_0__'],
        num_rows: 221
    })
})

In [7]:
def preprocess_function(examples):
    inputs = [text for text in examples["Text"]]
    # targets = ['' for bundles in examples["json"]]
    targets = [json_proc.process_json(bundles) for bundles in examples["json"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

ads = ads_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=ads_dataset["train"].column_names
)
ads = ads.flatten()

# ads_test = ads_test_dataset.map(
#     preprocess_function,
#     batched=True,
#     num_proc=4,
#     remove_columns=ads_test_dataset.column_names
# )
# ads_test = ads_test.flatten()


Map (num_proc=4):   0%|          | 0/8590 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/221 [00:00<?, ? examples/s]

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [9]:
from transformers import DataCollatorForSeq2Seq

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

In [10]:
from IPython.display import clear_output

class MetricComputer:
  def __init__(self):
    self.generations = []

  def __call__(self, eval_preds):
    ev = Evaluator(val_set, model=model, tokenizer=tokenizer, json_processor=json_proc)
    stats = ev.calc_bleu_batched()
    self.generations.append(ev.generate_samples_batched(count=20))
    # clear_output()
    return stats

In [11]:
n_epochs = 13

training_args = Seq2SeqTrainingArguments(
    output_dir="ruT5-large-sep-token",
    # overwrite_output_dir=True,
    evaluation_strategy="steps",
    eval_steps=1000,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=1000,
    num_train_epochs=n_epochs,
    # predict_with_generate=True,
    generation_max_length=128,
    fp16=True,
    lr_scheduler_type="cosine",
    group_by_length=False,
    warmup_steps=3,
    # load_best_model_at_end=True,
)

mc = MetricComputer()
empty_dataset = Dataset.from_dict({"Text": [], "json": []})
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=ads["train"],
    eval_dataset=ads["test"],
    # eval_dataset=empty_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=mc,
)

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/training_args.py:1493: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer.train()

Step,Training Loss,Validation Loss,Bep-sb,Bep-multi,Ta-bleu-sb,Ta-chrf-sb,Ta-chrf-multi,Eb-ind,Mb-ind,Bleu-classic,Chrf-classic,Chrf-classic-multi,Bleu Old,Failed Ratio
1000,0.525600,0.374192,0.684609,0.626046,38.860656,78.092665,75.328624,0.050000,0.064000,50.541831,75.717820,73.410428,50.541831,0.020000
2000,0.346000,0.361724,0.695174,0.635646,37.024365,77.384247,75.104369,0.098000,0.046000,51.479422,74.875147,73.698222,51.479422,0.022000
3000,0.239400,0.376220,0.700076,0.665405,39.653393,78.213808,77.129138,0.018000,0.072000,52.410314,75.239676,75.920694,52.410314,0.016000
4000,0.177600,0.393244,0.719664,0.685401,40.734752,79.587859,78.720500,0.022000,0.064000,53.176079,76.848018,76.640003,53.176079,0.008000
5000,0.132500,0.412297,0.720805,0.688442,41.223969,78.898236,77.829970,0.022000,0.058000,50.513002,75.667963,76.677035,50.513002,0.016000
6000,0.114300,0.417421,0.720871,0.687711,41.083173,79.182615,78.363664,0.024000,0.058000,52.179186,75.960376,76.317458,52.179186,0.008000


/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<T> кроссовки адидас озвиго оригинал<P> 150<C1> 1
<T> Часы Luxury (новые), 800<C1> 1<C2> RUB
<T> Нарды под старину, тематика Египет, черный мат в золотой патине, дома и фишки в морилке, черный мат в золотой патине, дома и фишки в морилке, черный мат в золотой патине, дома и фишки в морилке, черный мат в золотой патине, черный мат в золотой патине, черный мат в золотой патине, черный мат в золотой патине, черный мат в золотой патине, черный мат в золотой патине, дома и фишки в морилке, черный мат в золотой патине, черный мат в золотой патине, черный мат в золотой патине, черный мат в золотой патине, черный мат в золотой патине, черный мат
<T> Настольная игра Клуэдо/
<T> Кор
<T> футболка Gucci
<T> Сумка для ноутбука 15.6, цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфальт", цвет "мокрый асфа

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<T> Простыня на резинке, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикотаж, Трикот
<T> Спортивные штаны/джоггеры Befree
<T> Компьютер с процессором AMD Ryzen 5 3600, видеокартой GIGABTE RADEON RX 580 на 8 GB, оперативной памятью KINGSTON FR 16 GB (2x8GB) и SSD Kingston 4K, SSD Kingston 4K, SSD Kingston 4K, SSD Kingston 4K, SSD Kingston 4K, SSD Kingston 4K, SSD Kingston 4K, SSD Kingston 4K, SSD Kingston 4K, SSD Kingston 4K, SSD Kingston 4K
<T> mafia ps4<P> 10
<T> Камера Nikon 3200, 24MP CMOS, ISO 100-6400, 24MP CMOS, 24MP CMOS, 24MP CMOS, 24MP CMOS, 24MP CMOS, 24MP CMOS, 24MP CMOS, 24MP Hy1 Sensor, ISO 100-6400, Equivalent Hi1 Sensor, 24MP Hy1 Sensor, 24MP Hy1 Sensor, 24MP Hy1 Sense, 24P Hy1 Sense, 24P Hy
<T> Boomerang US - Eng / Fr<P>
<T> Комплект постельного белья от фирмы 'Пан
<

...

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<T> Простыня на резинке Трикотаж 100% хлопок
<T> Кожаная сумка с карманом Re
<T> расческа для пушистиков
<T>
<T> Сапоги резиновые, размер 22, новые<P> 400<C1>
<T> Платье (Турция),
<T> Ticket to ride: The Heart of Africa (настольная игра, настольная игра, настольная игра, настольная игра, настольная игра, настольная игра, настольная игра
<T> braddon «the lawyers secret»


/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<T> Простыня на резинке Трикотаж 100% хлопок
<T> расческа для пушистиков
<T> Платье (Турция),
<T> braddon «the lawyer


TrainOutput(global_step=6981, training_loss=0.27954767508862643, metrics={'train_runtime': 5881.738, 'train_samples_per_second': 18.986, 'train_steps_per_second': 1.187, 'total_flos': 5.8135359473664e+16, 'train_loss': 0.27954767508862643, 'epoch': 13.0})

In [13]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [20]:
tokenizer = AutoTokenizer.from_pretrained("./ruT5-large-sep-token/checkpoint-6981")
model = AutoModelForSeq2SeqLM.from_pretrained("./ruT5-large-sep-token/checkpoint-6981").to('cuda')

# output_dir = "ruT5-large-trained-sep-token"
# model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
mc(1)

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<T> Простыня на резинке Трикотаж 100% хлопок
<T> Платье новое (Турция), юбка пышная, Размер
<T>


{'BEP-sb': 0.7163310948712148,
 'BEP-multi': 0.686780074790732,
 'TA-BLEU-sb': 40.70550931579893,
 'TA-CHRF-sb': 78.82166839354579,
 'TA-CHRF-multi': 78.37031043374178,
 'EB-ind': 0.02,
 'MB-ind': 0.056,
 'BLEU-classic': 51.813201932893946,
 'CHRF-classic': 75.55203857816018,
 'CHRF-classic-multi': 76.57208671456245,
 'bleu_old': 51.813201932893946,
 'failed_ratio': 0.006}

In [25]:
ev = Evaluator(manual_test, model=model, tokenizer=tokenizer, batch_size=32, json_processor=json_proc)
output = ev.generate_samples_batched()
df = pd.DataFrame([{'id': i, 'json': json.dumps(v, indent=4, ensure_ascii=False)} for i, v in output.items()])
df.to_csv('manual_test_outputs_sep.csv', index=False)

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


<T> кресло компьютерное 30<C1> 1<C2> GEL


: 

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained(output_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained(output_dir).to('cuda')

In [ ]:
# distill_data = pd.read_csv('../data/distill_data.csv', index_col=0)
# distill_data.head()

In [ ]:
# ev = Evaluator(distill_data, model, tokenizer)
# output = ev.generate_samples_batched(batch_size=256)

In [ ]:
# distill_data['json'] = pd.Series(output)

In [ ]:
# distill_data.to_csv('../data/distill_100k.csv')